In [16]:
!pip install sentence-transformers

In [59]:
import pandas as pd
import numpy as np

from sentence_transformers import SentenceTransformer
from transformers import BertTokenizer, BertModel, BertModel,GPT2Tokenizer, GPT2Model, AutoTokenizer, AutoModelForCausalLM
import torch
import bertviz
from bertviz import head_view, model_view

In [18]:
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/Colab Notebooks/NLP/clean.xlsx'
df = pd.read_excel(path)
df.head()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,contract,paragraph,paragraph_title,paragraph_content,section,section_content,clean_paragraph_content,clean_section_content,paragraph_content_stemm,paragraph_content_lemma,paragraph_content_token,section_content_stemm,paragraph_section_lemma,paragraph_section_token
0,10,1.0,Vertragsgegenstand,1. Vertragsgegenstand 1.1 Diese Vertragsbeding...,1.1,1.1 Diese Vertragsbedingungen gelten für die N...,Diese Vertragsbedingungen gelten für die Nutzu...,Diese Vertragsbedingungen gelten für die Nutzu...,dies vertragsbedingungen gelten für die nutzun...,Diese Vertragsbedingungen gelten für die Nutzu...,"['dies', '##e', 've', '##rt', '##rag', '##sb',...",dies vertragsbedingungen gelten für die nutzun...,Diese Vertragsbedingungen gelten für die Nutzu...,"['dies', '##e', 've', '##rt', '##rag', '##sb',..."
1,10,1.0,Vertragsgegenstand,1. Vertragsgegenstand 1.1 Diese Vertragsbeding...,1.2,1.2 Die Software wird vom Anbieter als SaaS- b...,Diese Vertragsbedingungen gelten für die Nutzu...,Die Software wird vom Anbieter als SaaS bzw Cl...,dies vertragsbedingungen gelten für die nutzun...,Diese Vertragsbedingungen gelten für die Nutzu...,"['dies', '##e', 've', '##rt', '##rag', '##sb',...",die softwar wird vom anbiet al saa bzw cloudlö...,die Software wird vom Anbieter al saas bzw Clo...,"['die', 'software', 'wi', '##rd', 'vo', '##m',..."
2,10,1.0,Vertragsgegenstand,1. Vertragsgegenstand 1.1 Diese Vertragsbeding...,1.3,1.3 Diese Vertragsbedingungen gelten ausschlie...,Diese Vertragsbedingungen gelten für die Nutzu...,Diese Vertragsbedingungen gelten ausschließlic...,dies vertragsbedingungen gelten für die nutzun...,Diese Vertragsbedingungen gelten für die Nutzu...,"['dies', '##e', 've', '##rt', '##rag', '##sb',...",dies vertragsbedingungen gelten ausschließlich...,Diese Vertragsbedingungen gelten ausschließlic...,"['dies', '##e', 've', '##rt', '##rag', '##sb',..."
3,10,2.0,Art und Umfang der Leistung,2. Art und Umfang der Leistung Der Anbieter st...,no sections use paragraph,2. Art und Umfang der Leistung Der Anbieter st...,Der Anbieter stellt dem Kunden die Software in...,Der Anbieter stellt dem Kunden die Software in...,der anbiet stellt dem kunden die softwar in de...,Der Anbieter stellt dem Kunden die Software in...,"['der', 'an', '##bie', '##ter', 'ste', '##ll',...",der anbiet stellt dem kunden die softwar in de...,Der Anbieter stellt dem Kunden die Software in...,"['der', 'an', '##bie', '##ter', 'ste', '##ll',..."
4,10,3.0,Verfügbarkeit der Software,3. Verfügbarkeit der Software 3.1 Der Anbieter...,3.1,"3.1 Der Anbieter weist den Kunden darauf hin, ...",Der Anbieter weist den Kunden darauf hin dass ...,Der Anbieter weist den Kunden darauf hin dass ...,der anbiet weist den kunden darauf hin dass ei...,Der Anbieter weist den Kunden darauf hin dass ...,"['der', 'an', '##bie', '##ter', 'wei', '##st',...",der anbiet weist den kunden darauf hin dass ei...,Der Anbieter weist den Kunden darauf hin dass ...,"['der', 'an', '##bie', '##ter', 'wei', '##st',..."


# 1 Loading the model and tokenizer

In [19]:
bert_cased = BertTokenizer.from_pretrained('bert-base-cased')
model = BertModel.from_pretrained('bert-base-cased')


# 2. Appling the tokenizer on each of the cleand columns
This will give us a list of token-IDs for every toeken in every entity in each coumn as a list


In [20]:
df['tokens_paragraph_title'] = df['paragraph_title'].apply(lambda x: bert_cased.tokenize(str(x)))
df['bert_quick_paragraph_titel']=df['paragraph_title'].apply(lambda x: [bert_cased.encode(x)])
df['bert_quick_paragraph_content']=df['clean_paragraph_content'].apply(lambda x: [bert_cased.encode(x)])
df['bert_quick_section_content']=df['clean_section_content'].apply(lambda x: [bert_cased.encode(x)])
print("Clean content:")
print(df["paragraph_title"][0:10])
print("\n\n--> transformed to tokens (left out since we are using IDs):")
print(df['tokens_paragraph_title'][0:10])
print("\n\n--> transformed to Ids:")
print(df[["bert_quick_paragraph_titel"]][0:10])

Token indices sequence length is longer than the specified maximum sequence length for this model (590 > 512). Running this sequence through the model will result in indexing errors


Clean content:
0                              Vertragsgegenstand
1                              Vertragsgegenstand
2                              Vertragsgegenstand
3                     Art und Umfang der Leistung
4                      Verfügbarkeit der Software
5                      Verfügbarkeit der Software
6    Rechte zur Datenverarbeitung, Datensicherung
7    Rechte zur Datenverarbeitung, Datensicherung
8    Rechte zur Datenverarbeitung, Datensicherung
9    Rechte zur Datenverarbeitung, Datensicherung
Name: paragraph_title, dtype: object


--> transformed to tokens (left out since we are using IDs):
0     [V, ##ert, ##rag, ##s, ##ge, ##gens, ##tan, ##d]
1     [V, ##ert, ##rag, ##s, ##ge, ##gens, ##tan, ##d]
2     [V, ##ert, ##rag, ##s, ##ge, ##gens, ##tan, ##d]
3    [Art, und, Um, ##fan, ##g, der, Le, ##ist, ##ung]
4    [V, ##er, ##f, ##ü, ##g, ##bark, ##eit, der, S...
5    [V, ##er, ##f, ##ü, ##g, ##bark, ##eit, der, S...
6    [Re, ##cht, ##e, zur, Date, ##n, ##ver, ##ar, ...


Since Bert needs torch-tensors we simmply transform the lsit to torch tensors:


In [21]:
inputs_paragraph_title = df['bert_quick_paragraph_titel'].apply(lambda x: torch.tensor(x))
inputs_paragraph_content = df['bert_quick_paragraph_content'].apply(lambda x: torch.tensor(x))
inputs_section_content = df['bert_quick_section_content'].apply(lambda x: torch.tensor(x))
print(f'torch tensor for "{df["paragraph_title"][1]}":')
print(inputs_paragrpah_title[1])

torch tensor for "Vertragsgegenstand":
tensor([[  101,   159,  7340, 20484,  1116,  2176, 21144,  5108,  1181,   102]])


# 3. Structure of the model
## 3.1 Word embeddings

In [22]:
print(model.embeddings.word_embeddings)
print(f'--> {model.embeddings.word_embeddings.weight.shape[0]} tokens in the vocabulary')
print(f'--> {model.embeddings.word_embeddings.weight.shape[1]} dimensions per emabding verctor')

Embedding(28996, 768, padding_idx=0)
--> 28996 tokens in the vocabulary
--> 768 dimensions per emabding verctor


## 3.2 Position embeddings

In [23]:
print(model.embeddings.position_embeddings)
print(f'--> {model.embeddings.position_embeddings.weight.shape[0]} maximum chunk length')
print(f'--> {model.embeddings.position_embeddings.weight.shape[1]} dimensions per emabding verctor')

Embedding(512, 768)
--> 512 maximum chunk length
--> 768 dimensions per emabding verctor


## 3.3 Transformer Layers

In [24]:
print(len(model.encoder.layer))
print('--> Number of transformer layers')

12
--> Number of transformer layers


## 3.4 Structure of the first layer
TO BE DONE


In [25]:
print(model.encoder.layer[0])

BertLayer(
  (attention): BertAttention(
    (self): BertSdpaSelfAttention(
      (query): Linear(in_features=768, out_features=768, bias=True)
      (key): Linear(in_features=768, out_features=768, bias=True)
      (value): Linear(in_features=768, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (output): BertSelfOutput(
      (dense): Linear(in_features=768, out_features=768, bias=True)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (intermediate): BertIntermediate(
    (dense): Linear(in_features=768, out_features=3072, bias=True)
    (intermediate_act_fn): GELUActivation()
  )
  (output): BertOutput(
    (dense): Linear(in_features=3072, out_features=768, bias=True)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
)


## 3.5 Pooler
TO BE DONE

In [26]:
print(model.pooler)

BertPooler(
  (dense): Linear(in_features=768, out_features=768, bias=True)
  (activation): Tanh()
)


# 4. First run of BERT


In [36]:
with torch.no_grad():
 outputs = model(inputs_paragraph_title[0] , output_attentions=True)
 embeddings_all = outputs.last_hidden_state.numpy()
 embeddings = outputs.last_hidden_state[0] [0].numpy()

print(embeddings_all.shape)
print(f"--> 1st input has {embeddings_all.shape[0]} sentence and {embeddings_all.shape[1]} tokens with {embeddings_all.shape[2]}-dimanisonal embeddings")
print(embeddings.shape)
print(f"--> the CLS-Token (for the sentence) has a {embeddings.shape[0]}-dimensional embedding")


(1, 10, 768)
--> 1st input has 1 sentence and 10 tokens with 768-dimanisonal embeddings
(768,)
--> the CLS-Token (for the sentence) has a 768-dimensional embedding


<IPython.core.display.Javascript object>

# 4.1 Attentionsheads of BERT



In [42]:
attention_weights = outputs.attentions
head_view(attention_weights,
tokens=bert_cased.convert_ids_to_tokens(inputs_paragraph_title[0][0]))

<IPython.core.display.Javascript object>

## 4.2 Calculation of the CLS ebadings in BERT

In [43]:
def prepare_apply_model(tokenizer, text, model):
  input=torch.tensor([tokenizer.encode(text)])
  with torch.no_grad():
   outputs = model(input)
  embeddings = outputs.last_hidden_state[0][0].numpy()
  return embeddings

bert_1st_emb = prepare_apply_model(bert_cased, df["paragraph_title"].loc[0], model)
print("First 10 dimensions of the BERT-embedding of the first sentence:")
print(bert_1st_emb[0:9])

# 4. First run of SBERT

In [44]:
model_sbert_bert = SentenceTransformer("bert-base-nli-cls-token")


In [49]:

embedding = model_sbert_bert.encode(df["paragraph_title"][0], convert_to_tensor=True)

print(embedding.shape)
print(f"--> The sentence vector has {embedding.shape[0]} dimensionen.")

torch.Size([768])
--> The sentence vector has 768 dimensionen.


In [61]:
 def prepare_apply_model_SBERT(model, text):
  with torch.no_grad():
    embeddings = model.encode(text, convert_to_tensor=True)
  return embeddings
s_bert_1st_emb = prepare_apply_model_SBERT(model_sbert_bert, df["paragraph_title"].loc[0]).numpy()
print("First 10 dimensions of the BERT-embedding of the paragraph title:")
print(bert_1st_emb[0:9])
print("\n     VS.     \n\nFirst 10 dimensions of the SBERT-embedding of the first paragraph title:")
print(s_bert_1st_emb[0:9])
are_equal = np.array_equal(s_bert_1st_emb, bert_1st_emb)

print(f"\n\n-->The embeddings for BERT's sentence vector and SBERT's vector are {'equal' if are_equal else 'not equal'}!")

First 10 dimensions of the BERT-embedding of the first sentence:
[ 0.37478483  0.08961629 -0.12682848 -0.2669459  -0.05042043  0.10338283
  0.34543043 -0.09266926 -0.08774819]

     VS.     

First 10 dimensions of the SBERT-embedding of the first sentence:
[-0.47996977  0.06948641  0.96829104 -0.1758519   0.3606653   0.17929704
  0.37011206  0.66411424 -0.12230992]


-->The embeddings for BERT's sentence vector and SBERT's vector are not equal!


Since SBERT is Finetuned for recognizing semantic similarity for Sentences and paragraphs we want to use SBERT as a model. How ever we need to adjust the underlaying model so that it can fit better to german contract langauge.

# 5 Loading the GERMAN LEGAL model and tokenizer

In [77]:
legal_g_bert_cased = BertTokenizer.from_pretrained('deepset/gbert-base')
model_legal_g_bert = BertModel.from_pretrained('deepset/gbert-base')


# 7. Comparison GERMAN-LEGAL-BERT & generic BERT
This will give us a list of token-IDs for every toeken in every entity in each coumn as a list


In [72]:
df['tokens_paragraph_title_legal'] = df['paragraph_title'].apply(lambda x: legal_g_bert_cased.tokenize(str(x)))
df['legal_bert_quick_paragraph_titel']=df['paragraph_title'].apply(lambda x: [legal_g_bert_cased.encode(x)])
df['legal_bert_quick_paragraph_content']=df['clean_paragraph_content'].apply(lambda x: [legal_g_bert_cased.encode(x)])
df['legal_bert_quick_section_content']=df['clean_section_content'].apply(lambda x: [legal_g_bert_cased.encode(x)])
print("Clean content:")
display(df["paragraph_title"][0:10])
print("\n\n--> transformed to tokens (left out since we are using IDs):")
pd.set_option('display.max_colwidth', None)
display(df[[ 'tokens_paragraph_title_legal','tokens_paragraph_title']].head(10))
print("\n\n--> transformed to Ids:")
display(df[[ "legal_bert_quick_paragraph_titel","bert_quick_paragraph_titel"]].head(10))

Clean content:


,paragraph_title
0,Vertragsgegenstand
1,Vertragsgegenstand
2,Vertragsgegenstand
3,Art und Umfang der Leistung
4,Verfügbarkeit der Software
5,Verfügbarkeit der Software
6,"Rechte zur Datenverarbeitung, Datensicherung"
7,"Rechte zur Datenverarbeitung, Datensicherung"
8,"Rechte zur Datenverarbeitung, Datensicherung"
9,"Rechte zur Datenverarbeitung, Datensicherung"




--> transformed to tokens (left out since we are using IDs):


,tokens_paragraph_title_legal,tokens_paragraph_title
0,"[Vertrags, ##gegen, ##stand]","[V, ##ert, ##rag, ##s, ##ge, ##gens, ##tan, ##d]"
1,"[Vertrags, ##gegen, ##stand]","[V, ##ert, ##rag, ##s, ##ge, ##gens, ##tan, ##d]"
2,"[Vertrags, ##gegen, ##stand]","[V, ##ert, ##rag, ##s, ##ge, ##gens, ##tan, ##d]"
3,"[Art, und, Umfang, der, Leistung]","[Art, und, Um, ##fan, ##g, der, Le, ##ist, ##ung]"
4,"[Verfügbarkeit, der, Software]","[V, ##er, ##f, ##ü, ##g, ##bark, ##eit, der, Software]"
5,"[Verfügbarkeit, der, Software]","[V, ##er, ##f, ##ü, ##g, ##bark, ##eit, der, Software]"
6,"[Rechte, zur, Daten, ##verarbeitung, ,, Daten, ##sicherung]","[Re, ##cht, ##e, zur, Date, ##n, ##ver, ##ar, ##bei, ##tun, ##g, ,, Date, ##ns, ##iche, ##rung]"
7,"[Rechte, zur, Daten, ##verarbeitung, ,, Daten, ##sicherung]","[Re, ##cht, ##e, zur, Date, ##n, ##ver, ##ar, ##bei, ##tun, ##g, ,, Date, ##ns, ##iche, ##rung]"
8,"[Rechte, zur, Daten, ##verarbeitung, ,, Daten, ##sicherung]","[Re, ##cht, ##e, zur, Date, ##n, ##ver, ##ar, ##bei, ##tun, ##g, ,, Date, ##ns, ##iche, ##rung]"
9,"[Rechte, zur, Daten, ##verarbeitung, ,, Daten, ##sicherung]","[Re, ##cht, ##e, zur, Date, ##n, ##ver, ##ar, ##bei, ##tun, ##g, ,, Date, ##ns, ##iche, ##rung]"




--> transformed to Ids:


,legal_bert_quick_paragraph_titel,bert_quick_paragraph_titel
0,"[[102, 7545, 1789, 858, 103]]","[[101, 159, 7340, 20484, 1116, 2176, 21144, 5108, 1181, 102]]"
1,"[[102, 7545, 1789, 858, 103]]","[[101, 159, 7340, 20484, 1116, 2176, 21144, 5108, 1181, 102]]"
2,"[[102, 7545, 1789, 858, 103]]","[[101, 159, 7340, 20484, 1116, 2176, 21144, 5108, 1181, 102]]"
3,"[[102, 1622, 136, 7661, 125, 4970, 103]]","[[101, 2051, 5576, 12189, 16740, 1403, 4167, 3180, 1776, 4380, 102]]"
4,"[[102, 14967, 125, 5177, 103]]","[[101, 159, 1200, 2087, 17176, 1403, 24063, 27863, 4167, 10331, 102]]"
5,"[[102, 14967, 125, 5177, 103]]","[[101, 159, 1200, 2087, 17176, 1403, 24063, 27863, 4167, 10331, 102]]"
6,"[[102, 5487, 356, 1718, 13613, 818, 1718, 15295, 103]]","[[101, 11336, 9817, 1162, 23199, 14265, 1179, 4121, 1813, 16791, 19972, 1403, 117, 14265, 2316, 26312, 20901, 102]]"
7,"[[102, 5487, 356, 1718, 13613, 818, 1718, 15295, 103]]","[[101, 11336, 9817, 1162, 23199, 14265, 1179, 4121, 1813, 16791, 19972, 1403, 117, 14265, 2316, 26312, 20901, 102]]"
8,"[[102, 5487, 356, 1718, 13613, 818, 1718, 15295, 103]]","[[101, 11336, 9817, 1162, 23199, 14265, 1179, 4121, 1813, 16791, 19972, 1403, 117, 14265, 2316, 26312, 20901, 102]]"
9,"[[102, 5487, 356, 1718, 13613, 818, 1718, 15295, 103]]","[[101, 11336, 9817, 1162, 23199, 14265, 1179, 4121, 1813, 16791, 19972, 1403, 117, 14265, 2316, 26312, 20901, 102]]"


Since Bert needs torch-tensors we simmply transform the lsit to torch tensors:


In [76]:
inputs_paragraph_title_legal = df['legal_bert_quick_paragraph_titel'].apply(lambda x: torch.tensor(x))
inputs_paragraph_content_legal = df['legal_bert_quick_paragraph_content'].apply(lambda x: torch.tensor(x))
inputs_section_content_legal = df['legal_bert_quick_section_content'].apply(lambda x: torch.tensor(x))
print(f'Leagal torch tensor for "{df["paragraph_title"][1]}":')
print(inputs_paragraph_title_legal[1])

Leagal torch tensor for "Vertragsgegenstand":
tensor([[ 102, 7545, 1789,  858,  103]])


# 8. Structure of the legal german model
## 8.1 Word embeddings

In [78]:
print(model_legal_g_bert.embeddings.word_embeddings)
print(f'--> {model_legal_g_bert.embeddings.word_embeddings.weight.shape[0]} tokens in the vocabulary')
print(f'--> {model_legal_g_bert.embeddings.word_embeddings.weight.shape[1]} dimensions per emabding verctor')

Embedding(31102, 768, padding_idx=0)
--> 31102 tokens in the vocabulary
--> 768 dimensions per emabding verctor


## 8.2 Position embeddings

In [93]:
print(model_legal_g_bert.embeddings.position_embeddings)
print(f'--> {model.embeddings.position_embeddings.weight.shape[0]} maximum chunk length')
print(f'--> {model.embeddings.position_embeddings.weight.shape[1]} dimensions per emabding verctor')

Embedding(512, 768)
--> 512 maximum chunk length
--> 768 dimensions per emabding verctor


## 8.3 Transformer Layers

In [80]:
print(len(model_legal_g_bert.encoder.layer))
print('--> Number of transformer layers')

12
--> Number of transformer layers


## 8.4 Structure of the first layer
TO BE DONE


In [81]:
print(model_legal_g_bert.encoder.layer[0])

BertLayer(
  (attention): BertAttention(
    (self): BertSdpaSelfAttention(
      (query): Linear(in_features=768, out_features=768, bias=True)
      (key): Linear(in_features=768, out_features=768, bias=True)
      (value): Linear(in_features=768, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (output): BertSelfOutput(
      (dense): Linear(in_features=768, out_features=768, bias=True)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (intermediate): BertIntermediate(
    (dense): Linear(in_features=768, out_features=3072, bias=True)
    (intermediate_act_fn): GELUActivation()
  )
  (output): BertOutput(
    (dense): Linear(in_features=3072, out_features=768, bias=True)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
)


## 8.5 Pooler
TO BE DONE

In [82]:
print(model_legal_g_bert.pooler)

BertPooler(
  (dense): Linear(in_features=768, out_features=768, bias=True)
  (activation): Tanh()
)


# 9. First run of BERT


In [92]:
with torch.no_grad():
 outputs_legal = model_legal_g_bert(inputs_paragraph_title_legal[0] , output_attentions=True)
 embeddings_all_legal = outputs_legal.last_hidden_state.numpy()
 embeddings_legal = outputs_legal.last_hidden_state[0] [0].numpy()

print(embeddings_all_legal.shape)
print(f"--> 1st input has {embeddings_all_legal.shape[0]} sentence and {embeddings_all_legal.shape[1]} tokens with {embeddings_all_legal.shape[2]}-dimanisonal embeddings")
print(embeddings_legal.shape)
print(f"--> the CLS-Token (for the sentence) has a {embeddings_legal.shape[0]}-dimensional embedding")


(1, 5, 768)
--> 1st input has 1 sentence and 5 tokens with 768-dimanisonal embeddings
(768,)
--> the CLS-Token (for the sentence) has a 768-dimensional embedding


# 9.1 Attentionsheads of LEGAL G BERT



In [90]:
attention_weights = outputs_legal.attentions
head_view(attention_weights,
tokens=legal_g_bert_cased.convert_ids_to_tokens(inputs_paragraph_title_legal[0][0]))

<IPython.core.display.Javascript object>

## 9.2 Calculation of the CLS ebadings in LEGAL G BERT

In [95]:
legal_g_bert_1st_emb = prepare_apply_model(legal_g_bert_cased, df["paragraph_title"].loc[0], model_legal_g_bert )
print("First 10 dimensions of the LGAL G BERT`s Sentence embedding of the first paragraph title:")
print(legal_g_bert_1st_emb[0:9])
print("\n     VS.     \n\nFirst 10 dimensions of the genereic BERT`s Sentence embeddingof the first paragraph title:")
print(bert_1st_emb[0:9])

First 10 dimensions of the LGAL G BERT`s Sentence embedding of the first paragraph title:
[-0.5292427  -0.16948374 -0.06846911 -0.47662497 -0.7222663   0.06980199
  1.0041106   0.28140107 -0.01368893]

     VS.     

First 10 dimensions of the genereic BERT`s Sentence embeddingof the first paragraph title:
[ 0.37478483  0.08961629 -0.12682848 -0.2669459  -0.05042043  0.10338283
  0.34543043 -0.09266926 -0.08774819]
